In [1]:
# change dir
import os 
os.chdir('/work/jh2lab')

In [2]:
# get list of audio names to be transcribed

import glob
dir_list = glob.glob("/work/jh2lab/elic_model/elic_test/*.wav")
dir_list

['/work/jh2lab/elic_model/elic_test/ckm016-2023-06-10-Klana_11.wav',
 '/work/jh2lab/elic_model/elic_test/ckm015-2023-06-08-Klana_04.wav',
 '/work/jh2lab/elic_model/elic_test/ckm015-2023-06-08-Klana_03.wav',
 '/work/jh2lab/elic_model/elic_test/ckm016-2023-06-10-Klana_07.wav',
 '/work/jh2lab/elic_model/elic_test/ckm016-2023-06-10-Klana_09.wav',
 '/work/jh2lab/elic_model/elic_test/ckm016-2023-06-10-Klana_10.wav',
 '/work/jh2lab/elic_model/elic_test/ckm016-2023-06-10-Klana_08.wav',
 '/work/jh2lab/elic_model/elic_test/ckm016-2023-06-10-Klana_06.wav',
 '/work/jh2lab/elic_model/elic_test/ckm016-2023-06-10-Klana_01.wav',
 '/work/jh2lab/elic_model/elic_test/ckm009-2023-05-26-Žminj_01.wav',
 '/work/jh2lab/elic_model/elic_test/ckm015-2023-06-08-Klana_02.wav',
 '/work/jh2lab/elic_model/elic_test/ckm015-2023-06-08-Klana_05.wav',
 '/work/jh2lab/elic_model/elic_test/ckm016-2023-06-10-Klana_02.wav',
 '/work/jh2lab/elic_model/elic_test/ckm016-2023-06-10-Klana_05.wav',
 '/work/jh2lab/elic_model/elic_te

In [3]:
# get bench mark dataframe from textgrid file

import re
import pandas as pd
import numpy as np
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import torch


def get_audio_df(textgrid_dir):
    
    #with open(textgrid_dir, 'r', encoding='utf-16') as file:
    #with open(textgrid_dir, 'r', encoding='utf-8') as file:
        #lines = [line.rstrip() for line in file]
    try:
        with open(textgrid_dir, 'r', encoding='utf-8') as f:
            lines = [line.rstrip() for line in f]
    except:
        with open(textgrid_dir, 'r', encoding = 'utf-16-be') as f:
            lines = [line.rstrip() for line in f]   
    text_list = []
    xmin_list = []
    xmax_list = []
    interval_list = []
    for line in lines[14:]:  #informations needed begin on the 9th lines
        if 'text =' in line:
            line = line.split('"')[1]
            if line != "":
                text_list.append(line)
                #print (len(text_list), len(xmin_list), len(xmax_list), len(interval_list), line, interval_list[-2:])
            else:
                xmin_list = xmin_list[:-1]
                xmax_list = xmax_list[:-1]
                interval_list = interval_list[:-1]
        if 'xmin' in line:
            time_min = line.split('=')[1].strip(' ')
            xmin_list.append(time_min)
        if 'xmax' in line:
            time_max = line.split('=')[1].strip(' ')
            xmax_list.append(time_max)
        if 'intervals [' in line:
            interval = line.split('[')[1].strip(']:')
            interval_list.append(interval)
        if 'intervals: size' in line:
            xmin_list = xmin_list[:-1]
            xmax_list = xmax_list[:-1]
    xmin_list = [float(item) for item in xmin_list]
    xmax_list = [float(item) for item in xmax_list]
    interval_list = [int(item) for item in interval_list]

    df = pd.DataFrame()
    df['text'] = text_list
    df['xmin'] = xmin_list
    df['xmax'] = xmax_list
    df['interval'] = interval_list
    df['tier'] = get_interval(df['interval'].tolist())
    df['start'] = df['xmin'].apply(lambda x: int(np.floor(x*16000)))
    df['end'] = df['xmax'].apply(lambda x: int(np.ceil(x*16000)))
    return df

def get_interval(time_list):
    interval_all = []
    interval_curr = 1
    for i in range(len(time_list)):
        if i == 0:
            interval_all.append(1)
        else:
            if time_list[i] > time_list[i - 1]:
                interval_all.append(interval_curr)
                continue
            else:
                interval_curr = interval_curr + 1
                interval_all.append(interval_curr)
    return interval_all

def locate_end_small(xmin_list, xmax_list, i):
    start_curr = xmin_list[i]
    end_curr = xmax_list[i]

    if xmax_list[-1] - start_curr <= 10: # if the end of max < 15, return the last xmax
        return xmax_list[-1], len(xmin_list) + 1

    if end_curr - start_curr >= 10:
        return end_curr, i + 1
    
    while i < len(xmin_list):
        end_curr = xmax_list[i]
        #print ('----', start_curr, end_curr, i)
        if end_curr - start_curr >= 10:
            
            if xmin_list[i] - start_curr >= 10: # current start - last end > 10 blank, sep the blank and extend the last one
                return start_curr + 8,i
            else:
                return end_curr, i + 1
      
        else:
            i = i + 1
            continue
            

def get_chunk_text(start_id, end_id, file_tier, df):
    df_curr = df[df['tier'] == file_tier] 
    text_list = df_curr['text'].tolist()
    text_chunks = text_list[start_id : end_id]
    text = ' '.join(text_chunks)
    return text

In [4]:
def get_chunk_df(textgrid_dir):
    df = get_audio_df(textgrid_dir)
    df_12 = df[df['tier'].isin([1,2])]
    df_12['length'] = df_12['xmax'] - df_12['xmin']
    file_name = textgrid_dir.split('/')[-1].split('.')[0]
    start = []
    end = []
    start_id = []
    end_id = []
    file_info = []
    chunk_name = []

    for tier in [1,2]:
        #print (tier)

        df_curr = df_12[df_12['tier'] == tier]
        xmin_list = df_curr['xmin'].tolist()
        xmax_list = df_curr['xmax'].tolist()
    
        i = 0
        j = 0 # counting the file chunk number
        while i < len(xmin_list):
            start_curr = xmin_list[i]
            i_curr = i
            # locate the one that > 15 and  < 25 in endlist
            end_curr = locate_end_small(xmin_list, xmax_list, i)[0]
            i = locate_end_small(xmin_list, xmax_list, i)[1]
            #print (start_curr, end_curr, i_curr, i, end_curr-start_curr)
        
            j = j + 1 # chunk number update
        
            start.append(start_curr)
            end.append(end_curr)
            start_id.append(i_curr)
            end_id.append(i)
            file_info.append(tier)
            chunk_name.append(file_name + '_' + str(j))
    df_chunk = pd.DataFrame()
    df_chunk['chunk_name'] = chunk_name
    df_chunk['file_tier'] = file_info
    df_chunk['start'] = start
    df_chunk['end'] = end
    df_chunk['start_id'] = start_id
    df_chunk['end_id'] = end_id
    df_chunk['text'] = df_chunk.apply(lambda x: get_chunk_text(x.start_id, x.end_id, x.file_tier, df), axis = 1)
    return df_chunk

In [5]:
from transformers import pipeline
pipe_1 = pipeline(model="ninninz/whisper-ckm-1") 
pipe_2 = pipeline(model="ninninz/whisper-ckm-2") 
pipe_3 = pipeline(model="ninninz/whisper-ckm-3") 
pipe_4 = pipeline(model="ninninz/whisper-large-v3-croatian-10") 
pipe_5 = pipeline(model="ninninz/whisper-large-v3-croatian-20") 
pipe_6 = pipeline(model="ninninz/whisper-large-v3-croatian-30") 
pipe_7 = pipeline(model="ninninz/whisper-large-v3-croatian-overlap-removed-10") 
pipe_8 = pipeline(model="ninninz/whisper-large-v3-croatian-overlap-removed-20") 
pipe_9 = pipeline(model="ninninz/whisper-large-v3-croatian-overlap-removed-30") 
pipe_10 = pipeline(model="openai/whisper-large-v3") 

2024-04-14 21:33:33.087308: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-14 21:33:33.090234: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-14 21:33:33.091135: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-14 21:33:33.101884: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-14 21:33:34.809146: W tensorflow/compiler/tf2

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
def get_whisper_text(t1, t2, speech, model):
    t1 = int(t1 * 16000) #Works in milliseconds
    t2 = int(t2 * 16000)
    newAudio = speech[t1:t2]
    result = model(newAudio, generate_kwargs={"language": "croatian"})['text']
    return result


In [7]:
import librosa
def get_all_transcription(audio_dir, df_chunk):
    speech, sample_rate = librosa.load(audio_dir, sr=16000)
    df_chunk['transcription_v1'] = df_chunk.apply(lambda x: get_whisper_text(x.start, x.end, speech, pipe_1), axis = 1)
    print ('1')
    df_chunk['transcription_v2'] = df_chunk.apply(lambda x: get_whisper_text(x.start, x.end, speech, pipe_2), axis = 1)
    print ('2')
    df_chunk['transcription_v3'] = df_chunk.apply(lambda x: get_whisper_text(x.start, x.end, speech, pipe_3), axis = 1)
    print ('3')
    df_chunk['transcription_v4'] = df_chunk.apply(lambda x: get_whisper_text(x.start, x.end, speech, pipe_4), axis = 1)
    print ('4')
    df_chunk['transcription_v5'] = df_chunk.apply(lambda x: get_whisper_text(x.start, x.end, speech, pipe_5), axis = 1)
    print ('5')
    df_chunk['transcription_v6'] = df_chunk.apply(lambda x: get_whisper_text(x.start, x.end, speech, pipe_6), axis = 1)
    print ('6')
    df_chunk['transcription_v7'] = df_chunk.apply(lambda x: get_whisper_text(x.start, x.end, speech, pipe_7), axis = 1)
    print ('7')
    df_chunk['transcription_v8'] = df_chunk.apply(lambda x: get_whisper_text(x.start, x.end, speech, pipe_8), axis = 1)
    print ('8')
    df_chunk['transcription_v9'] = df_chunk.apply(lambda x: get_whisper_text(x.start, x.end, speech, pipe_9), axis = 1)
    print ('9')
    df_chunk['transcription_v10'] = df_chunk.apply(lambda x: get_whisper_text(x.start, x.end, speech, pipe_10), axis = 1)
    print ('10')
    file_name = audio_dir.split('/')[-1].split('.')[0]
    print (file_name)
    df_chunk.to_csv('/home/sz38235/workDir/elic/whisper_test/' + file_name + '_whisper_all.csv')
    return df_chunk

In [16]:
i = 2
audio_dir = dir_list[i]
textgrid_dir = dir_list[i].split('.wav')[0] + '.TextGrid'
print (textgrid_dir)

df_chunk = get_chunk_df(textgrid_dir)
df_chunk = get_all_transcription(audio_dir, df_chunk)
df_chunk

/work/jh2lab/elic_model/elic_test/ckm015-2023-06-08-Klana_03.TextGrid
1
2
3
4
5
6
7
8
9
10
ckm015-2023-06-08-Klana_03


,chunk_name,file_tier,start,end,start_id,end_id,text,transcription_v1,transcription_v2,transcription_v3,transcription_v4,transcription_v5,transcription_v6,transcription_v7,transcription_v8,transcription_v9,transcription_v10
0,ckm015-2023-06-08-Klana_03_1,1,0.772180,11.165106,0,2,Se domišljate neke smiješne zgode možda ili ne...,se domišljate neke smiješne zgode možda ili ne...,Se domišljate neke smiješne zgode možda ili ne...,se domišljate neke smiješne zgode možda ili ne...,Se domišljate neke smiješne zgode možda ili ne...,Se domišljate neke smiješne zgode možda il neč...,"Se domišljate neke smiješne zgode, možda, ili ...","Se domišljate neke smiješne zgode, možda, ili ...","Se domišljate neke smiješne zgode, možda, ili ...","Se domišljate neke smiješne zgode, možda, ili ...",Se domišljate neke smiješne zgode možda ili n...
1,ckm015-2023-06-08-Klana_03_2,1,11.165106,19.165106,2,4,((breath)) A kako je bilo za @ krštenje ili @?,A kako je bilo za krštenje ili?,A kako je bilo za krštenje ili-,A kako je bilo za Krštenje ili?,"A kako je bilo za kraštenje, ovi, eh, sad se t...",A kako je bilo kad kuhala možda?,A kako je bilo za Krštenje ili?,A kako je bilo za kršten ovi ki su,A kako je bilo za Krštenje ili...?,A kako je bilo za Krštenje?,"A kako je bilo za krštenje? Za krštenje, sad ..."
2,ckm015-2023-06-08-Klana_03_3,1,46.988200,54.988200,4,5,Mhm.,"A mislin da je, ovaj, nezor, nezor, prvo, a pr...",Velimi. A ovi- i mislin da je kul maj- kul par...,Mhm.,Mhm.,M.,Mhm.,Mhm.,Mhm.,"Velimi. A ovi... I mislin da je , da je kumaj...",Mislim da je kum par suprinesli djecu krstiti.
3,ckm015-2023-06-08-Klana_03_4,1,61.620838,75.614588,5,9,"Mhm. To je bilo tako (raneje), mhm. ((breath))...",Hm. Ča je bilo tako? A recite mi čime su se ju...,"To je bilo tako. A retite mi, čima su se ljudi...",Mhm. To je bilo tako.,Mhm. Jako.,Mhm. Ja.,Mhm. Ja.,Mhm. Da. A retite mi čime su se ljudi prije ba...,Mhm.,Mhm. Ja.,To je bilo tako radno. A čime su se ljudi pri...
4,ckm015-2023-06-08-Klana_03_5,1,75.614588,83.614588,9,12,((breath)) Jer nekako mi se čini da Klana- mhm...,"Pa, moji stariji, ja, moj nono, va biti, va kl...","Pa, mojii stariji, ja, moj nono, va biti, va k...","Pa, moji stariji, ja, moj nono, va biti, va kl...","Pa moji stariji, ja, moj nono, va biti, va kla...","A, moji stariji... Ja, moj nono, va biti va kl...",A ča se svega drugoga? Č,"Pa moji stariji. Ja, moj nono va biti, va klan...","Pa, moji stariji, ja, moj nono, va biti, va kl...","A, moji stariji, ja, moj nono, va biti, va kla...","Moji stariji, moj nono, vabiti Vaklani je bil..."
5,ckm015-2023-06-08-Klana_03_6,1,159.080007,167.080007,12,13,Mhm.,Mhm.,Mhm.,Mhm.,Mhm.,Mhm.,Mhm.,Mhm.,Mhm.,Mhm.,"To su vozili u Riku, kamo su morali. Tako da ..."
6,ckm015-2023-06-08-Klana_03_7,1,189.269668,197.269668,13,14,Mhm.,"A ča se dela? Je bilo, ali...","ilat, onda su te, prije su bile ciste, ni bilo...","i onda su te prije su bile ciste, ni bilo asfa...",Mhm.,Pa dole.,Mhm.,Tako je va Vrbniku. Evo va va va va va va va v...,Tako. I onda to bilo aš je bilo teta isto ško...,"A,","Bila su ciste, nije bilo asfalta. To je onda ..."
7,ckm015-2023-06-08-Klana_03_8,1,203.378479,211.378479,14,17,"Mhm. A kako sad mjesto izgleda i što je sad, k...",Hm. Kako sad za mjesto?,M? Kako sad za mjesto izgleda i š to je sad ka...,"M? Kako sad za mjesto izgleda i što je sad, ka...",Mhm. Kako je sad za mjesto izgleda i što to je...,Mhm. A kako je sad za mjesto? Za štstosto?,Mhm. Kako sad za mjesto?,"Mhm. Kako je sad za mjesto i što je sad, kako ...","Mhm. Kako sad za mjesto izgleda i što je sad, ...",M? Kako je sad za mjesto?,Kako je sad za mjesto izgleda i čime se ljudi...
8,ckm015-2023-06-08-Klana_03_9,1,237.116762,247.798831,17,20,Mhm. ((breath)) A- a recite mi @- kad @- govor...,"A recite mi, kad govorite ovo mjestu, rekli st...","A, recite mi, kad govorite o mjestu, rekli ste...","Mhm. A recite mi, kad govorite o v mjestu rekl...","Mhm. A recite mi, kad govorite o mjestu, rekli...","Mhm. A recite mi kad.)hhh.) govorite o

In [ ]:
# do all rest audio files in the list

for i in range(11, 13):
    print (i)
    audio_dir = dir_list[i]
    textgrid_dir = dir_list[i].split('.wav')[0] + '.TextGrid'
    print (textgrid_dir)
    df_chunk = get_chunk_df(textgrid_dir)
    df_chunk = get_all_transcription(audio_dir, df_chunk)

11
/work/jh2lab/elic_model/elic_test/ckm015-2023-06-08-Klana_05.TextGrid
1
2
3
4
